In [124]:
from __future__ import print_function
import ipywidgets as widgets
from ipywidgets import GridspecLayout, Button, Layout
import plotly.graph_objects as go
import folium
import warnings
from folium.plugins import HeatMapWithTime
import pickle
import numpy as np
import pandas as pd
from datetime import datetime

warnings.filterwarnings("ignore")

In [125]:
# This widget displays pedestrian traffic image on the webpage
file = open("pedestrianTraffic.jpg", "rb")
image = file.read()
pedsImage=widgets.Image(
    value=image,
    format='jpg',
    layout=Layout(width='100%', height='500px')
)
display(pedsImage)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xfe\x00;CREATOR: gd-jpeg v1.0…

In [126]:
#This widget displays information about the webpage
info = widgets.Button(
    description='This tool predicts pedestrian and cyclist traffic at selected intersections at Downtown Toronto',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    layout=Layout(width='100%', height='40px')
)
display(info)

Button(button_style='info', description='This tool predicts pedestrian and cyclist traffic at selected interse…

In [127]:
# Reading saved models; these models implement Gaussian process regression on pedestrian and cyclist data
filename = 'gprPedsMdl.pickle'
gprPedsMdl = pickle.load(open(filename, 'rb'))

filename = 'gprCycsMdl.pickle'
gprCycsMdl = pickle.load(open(filename, 'rb'))

ToLocations = pd.read_csv('miovision_intersections.csv', low_memory = False);
Locations = ToLocations[['lat','lng']];

In [128]:
#Hoidays in Toronto in 2019; That a given day is a holiday was found to be indicative of pedestrian/ cylcist traffic

holidayDates = ['2019-01-01', '2019-02-14', '2019-02-18', '2019-03-17',
          '2019-04-19', '2019-04-22', '2019-05-12', '2019-05-20',
          '2019-06-16', '2019-07-01', '2019-08-05', '2019-09-02',
          '2019-10-14', '2019-10-31', '2019-12-25', '2019-12-26']

holidayDates = pd.to_datetime(holidayDates, dayfirst = True)
HolidayCal = pd.DataFrame(holidayDates, columns = ['dates']);

In [129]:
IntersectionDict = {'Adelaide-Bay': 0, 'Adelaide-Jarvis': 1, 'Front-Bathurst': 2, 'King-Portland': 3,
                   'King-Peter': 4, 'King-Bay': 5, 'King-Yonge': 6, 'King-Church': 7, 'King-Jarvis': 8,
                   'Queen-Bathurst': 14, 'Richmond-Spadina': 9, 'Richmond-Bay': 10, 'Wellington-Blue_Jays': 11,
                   'Wellington-Bay': 12, 'King-John': 13}

In [130]:
#User inputs are saved as widget outputs: these need to be converted to the appropriate data types for the functions..
# get_forecast() and get_maps(), which implement forecasting based on Gaussian process regression

def get_param(ForecastDate,StartHour,forecastHorizon,InterSection,RoadUser):
    
    FirstDate= ForecastDate.value #This is the first date for which a user wants forecast

    FirstTime=int(StartHour.value); #This is the first hour for which a user wants forecast on FirstDate

    Horizon=forecastHorizon.value; #This is how long in the future (in hours) a user wants traffic forecasts,
    #starting from FirstDate+FirstTime

    testStart = datetime(FirstDate.year, FirstDate.month, FirstDate.day)
    testStart = testStart + pd.Timedelta(hours=FirstTime);
    
    testEnd = testStart + pd.Timedelta(hours=Horizon-1);
    
    IntID = IntersectionDict[InterSection.value]
        
    if RoadUser.value == 'Pedestrians':
        RoadUserID = 1;
    elif RoadUser.value == 'Cyclists':
        RoadUserID = 2;
        
    return RoadUserID, InterSection.value, IntID, testStart, testEnd, Horizon

In [307]:
def get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon):
    
    if RoadUserID == 1:
        gprMdl = gprPedsMdl
    elif RoadUserID == 2: 
        gprMdl = gprCycsMdl
    
    dateTimeIdx = pd.date_range(start = testStart, periods = Horizon, freq = 'H');
      
    hour = dateTimeIdx.hour; dayOfWeek = dateTimeIdx.dayofweek; month = dateTimeIdx.month
    
    dates = pd.DataFrame(dateTimeIdx.date, columns=['dates']);
    dates = dates['dates']
    Holidays = dates.isin(HolidayCal['dates'].dt.date)
    Holidays[Holidays == True] = 1; Holidays[Holidays == False]=0;
    
    lat = Locations.iloc[IntID, 0]; lng = Locations.iloc[IntID, 1]
    
    #Creating the input vector to the Gaussian process regression model, which is made up of: hour, day of week,
    # binary holiday variable, latitude and longitude
    xTest = pd.DataFrame();
    xTest.insert(0, 'Hour', hour);
    xTest.insert(1, 'DayOfWeek', dayOfWeek)
    xTest.insert(2, 'isHoliday', Holidays)
    xTest.insert(3, 'lat', lat*np.ones([len(xTest),1]));
    xTest.insert(4, 'lng', lng*np.ones([len(xTest),1]))
       
    xTest.index = dateTimeIdx;
    
    #Prediction with Gaussian process regression
    yPred, yStd = gprMdl.predict(xTest, return_std=True)
    yPred = yPred.astype('int'); yPred[yPred<0]=0
    yPred = pd.DataFrame(yPred, columns =['forecast']); yPred.index = dateTimeIdx
    yPred = yPred['forecast']
    
    #We take +/- 3 times the standard deviation representing a 99.7% confidence bound
    yLow = yPred - 3*yStd; yLow = yLow.astype('int'); yLow[yLow<0] = 0
    yHigh = yPred + 3*yStd; yHigh = yHigh.astype('int');
    
    #Plotting the forecast as well as its confidence bound
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yPred, name = 'Forecast'))
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yLow, 
                            mode = 'lines',
                            line = dict(width = 0.1),
                            name = ''))
    fig.add_trace(go.Scatter(x = dateTimeIdx, y = yHigh, fill = 'tonexty', 
                            mode = 'lines',
                            line = dict(width=0.1),
                            name = '99.7% confidence interval'))
    fig.update_xaxes(title_text = 'Time')
    if RoadUserID == 1: 
        fig.update_yaxes(title_text='Pedestrian volume')
        fig.update_layout(width=1400, height=700, 
                          title_text = 'Pedestrian traffic forecast at '+ Intersection + ' Intersection')
    elif RoadUserID == 2:
        fig.update_yaxes(title_text='Cyclist volume')
        fig.update_layout(width=1400, height=700,
                          title_text = 'Cyclist traffic forecast at '+ Intersection + ' Intersection')
    fig.show()

In [308]:
def get_maps(RoadUserID, IntID, testStart, testEnd, Horizon):
    
    if RoadUserID == 1:
        gprMdl = gprPedsMdl
    elif RoadUserID == 2: 
        gprMdl = gprCycsMdl
    
    IntNames = ['Adelaide-Bay', 'Adelaide-Jarvis', 'Front-Bathurst', 'King-Portland',
               'King-Peter', 'King-Bay', 'King-Yonge', 'King-Church', 'King-Jarvis',
               'Richmond-Spadina','Richmond-Bay','Wellington-Blue_Jays',
                'Wellington-Bay','King-John','Queen-Bathurst'];
    
    IntLoc = Locations.iloc[13,:] #On running the script, Location 13 is shown on map of Toronto; this is selected 
    #because it is the middlemost-located intersection that allows easy visualisation of all other intersections
    
    #In the following lines, we create a map visualisation using Folium package; this visualisation shows a heatmap
    #of pedestrian/ cyclist traffic and how they evolve with time at all the intersections in the dataset, 
    #within the forecast horizon specified
    
    trafficMap = folium.Map(location= IntLoc, zoom_start = 15)
    
    dateTimeIdx = pd.date_range(start = testStart, periods = Horizon, freq = 'H');
      
    hour = dateTimeIdx.hour; dayOfWeek = dateTimeIdx.dayofweek; month = dateTimeIdx.dayofweek;
    
    dates = pd.DataFrame(dateTimeIdx.date, columns=['dates']);
    dates = dates['dates']
    Holidays = dates.isin(HolidayCal['dates'].dt.date)
    Holidays[Holidays == True] = 1; Holidays[Holidays == False]=0;
    

    
    IntForecasts = [ [] for i in range(Horizon) ]
    for IntID in range(len(Locations)):

        lat = Locations.iloc[IntID, 0]; lng = Locations.iloc[IntID, 1]
        
        xTest = pd.DataFrame();  
        xTest.insert(0, 'Hour', hour);
        xTest.insert(1, 'DayOfWeek', dayOfWeek)
        xTest.insert(2, 'isHoliday', Holidays)
        xTest.insert(3, 'lat', lat*np.ones([len(xTest),1]));
        xTest.insert(4, 'lng', lng*np.ones([len(xTest),1]));     
        xTest.index = dateTimeIdx;

        yPred, yStd = gprMdl.predict(xTest, return_std=True)
        yPred = yPred.astype('int'); yPred[yPred<0]=0
        yPred = pd.DataFrame(yPred, columns =['forecast']); yPred.index = dateTimeIdx;
        yPred = yPred['forecast']
        
        IntLoc = Locations.iloc[IntID,:]
        
        folium.Marker(location = IntLoc, popup = IntNames[IntID]).add_to(trafficMap) 
        
        for i in range(Horizon):
            IntForecasts[i].append([IntLoc.values[0], IntLoc.values[1], yPred.values[i]])
    
    HeatMapWithTime(IntForecasts, radius=18, auto_play = True,
                    gradient={0: 'blue', 0.2: 'lime', 0.4: 'orange', 0.6: 'red', 1: 'black'}, 
                    min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(trafficMap)
    display(trafficMap)

In [309]:
grid = GridspecLayout(1, 4, grid_gap="10px")

In [310]:
#This widget is a dropdown for taking in the intersection of choice
InterSection = widgets.Dropdown(
    options = ['Adelaide-Bay', 'Adelaide-Jarvis','Front-Bathurst', 
             'King-Bay', 'King-Church', 'King-Jarvis',
             'King-John', 'King-Peter', 'King-Portland', 
             'King-Yonge', 'Queen-Bathurst', 'Richmond-Bay',
             'Richmond-Spadina', 'Wellington-Bay', 'Wellington-Blue_Jays'],
    description = 'Intersection:',
    value = 'King-John',
    disabled = False,
)
grid[0,1] = InterSection

In [311]:
#This is another dropdown widget for taking in the road user of choice -- pedestrians or cyclists
RoadUser = widgets.Dropdown(
    options = ['Pedestrians', 'Cyclists'],
    description = 'Road user:',
    value = 'Cyclists',
    disabled = False,
)
grid[0,2] = RoadUser

In [312]:
display(grid)

GridspecLayout(children=(Dropdown(description='Intersection:', index=6, layout=Layout(grid_area='widget001'), …

In [313]:
grid = GridspecLayout(1, 3, grid_gap = "10px")

In [314]:
#This widget allows the user to input the forecast start date
style = {'description_width': 'initial'}
ForecastDate = widgets.DatePicker(
    style = style,
    description = 'Forecast start date:',
    value = pd.to_datetime('2019-08-01'),
    disabled = False
)
grid[0,0] = ForecastDate

In [315]:
#This widget allows the user to input the forecast start hour
style = {'description_width': 'initial'}
StartHour = widgets.Dropdown(
    options = ['0','1', '2', '3','4','5','6','7','8','9','11','12',
            '13','14','15','16','17','18','19','20','21','22','23'],
    style = style,
    description = 'Forecast start hour:',
    value = '0',
    disabled = False,
)
grid[0,1] = StartHour

In [316]:
#This widget allows the user to specify how long in the future in hours for which they want to obtain forecasts
style = {'description_width': 'initial'}
forecastHorizon = widgets.IntText(
    value = 24,
    style = style,
    description = 'Forecast horizon in hours:',
    disabled = False
)
grid[0,2] = forecastHorizon

In [317]:
display(grid)

GridspecLayout(children=(DatePicker(value=Timestamp('2019-08-01 00:00:00'), description='Forecast start date:'…

In [318]:
grid = GridspecLayout(1, 4, grid_gap = "10px")

In [319]:
# This widget receives a button click on the webpage, and uses that click signal to call the function responsible for
# evaluating the forecasts
style = {'description_width': 'initial'}
ts_button = widgets.Button(description = "Click here for graph",
                          button_style = 'success')
ts_output = widgets.Output()
grid[0,1] = ts_button

def on_ts_button_clicked(b):
    with ts_output:
        print('This might take a while...')
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        
        get_forecast(RoadUserID, Intersection, IntID, testStart, testEnd, Horizon)
        print('Completed!')
        ts_output.clear_output(wait = True)
        
ts_button.on_click(on_ts_button_clicked)

In [320]:
# This widget receives a button click on the webpage, and uses that click signal to call the function responsible for
# generating the heatmaps
style = {'description_width': 'initial'}
maps_button = widgets.Button(description = "Click here for map",
                            button_style = 'success',
                            style = style)
maps_output = widgets.Output()
grid[0,2] = maps_button

def on_maps_button_clicked(b):
    with maps_output:
        print('This might take a while...')
        RoadUserID, Intersection, IntID, testStart, testEnd, Horizon = \
        get_param(ForecastDate, StartHour, forecastHorizon, InterSection, RoadUser)
        get_maps(RoadUserID, testStart, testStart, testEnd, Horizon)
        print('Completed!')
        maps_output.clear_output(wait = True)
        
maps_button.on_click(on_maps_button_clicked)

In [321]:
#This is an information widget, directing the user to click for either forecasts or for heatmaps
info = widgets.Button(
    description = 'Click below for time-series forecast graph, or heatmap of traffic volumes evolving with time',
    disabled = True,
    button_style = '',
    layout = Layout(width = '100%', height = '40px')
)
display(info)

Button(description='Click below for time-series forecast graph, or heatmap of traffic volumes evolving with ti…

In [322]:
display(grid)
display(ts_output)
display(maps_output)

GridspecLayout(children=(Button(button_style='success', description='Click here for graph', layout=Layout(grid…

Output()

Output()